In [ ]:
import pandas as pd
import numpy as np
import xpress as xp
import os
import zipfile

# Initialize Xpress
xp.init('C:/xpressmp//bin/xpauth.xpr')

# =============================================================================
# 1. Data Loading (Keep unchanged, read data only once at the outermost level)
# =============================================================================
VehicleCapacity = {1: 9.0, 2: 2.4, 3: 1.5}
VehicleCostPerMileAndTonneOverall = {1: 0.185, 2: 0.720, 3: 0.857}

df1 = pd.read_csv('Sliced_Candidate_Supplier_Distance.csv')
Dist_Sup_to_Warehouse = df1.values

df2 = pd.read_csv('Sliced_Candidate_Customer_Distance.csv')
Dist_Warehouse_to_Cust = df2.values

df3 = pd.read_excel('New_Warehouses_Costs_Capacity.xlsx')
Base_TotalSetupCost = df3['Total Setup Cost'].values 
Base_OperatingCost = df3['Operating Cost'].values 
Base_TotalCapacity = df3['Total Capacity'].values

df4 = pd.read_csv('Demand.csv')
demand_matrix_df = df4.pivot(index='Product', columns='Customer', values='Demand').fillna(0)
demand_matrix = demand_matrix_df.values

df5 = pd.read_csv('Suppliers.csv')
supplier_cap_matrix_df = df5.pivot(index='ID', columns='Product group', values='Capacity').fillna(0)
Base_supplier_cap_matrix = supplier_cap_matrix_df.values

df6 = pd.read_csv('Suppliers.csv')
Vehicle = df6['Vehicle type'].values 
Product_Sup = df6['Product group'].astype(int).tolist()

demand_matrices = {}
for t in range(1, 11):
    df8 = pd.read_excel(f"Demand_Period_{t}.xlsx", header=None)
    demand_matrices[t] = df8.values

Groups = range(Dist_Sup_to_Warehouse.shape[0])
Supplier = range(Dist_Sup_to_Warehouse.shape[1])
Customer = range(df8.shape[1])
Period = range(10)
Products = range(demand_matrix.shape[0])

# =============================================================================
# 2. Base Transport Cost Calculations (Compute base transport cost, later multiplied by α)
# =============================================================================
Base_CostSupplierWarehouse = np.zeros_like(Dist_Sup_to_Warehouse)
for j in Groups:
    for k in Supplier:
        # [Bug Fix 1] Previously written as Vehicle[i], should be Vehicle[k] (each supplier corresponds to a vehicle type)
        v_type = Vehicle[k]
        Base_CostSupplierWarehouse[j, k] = 2 * Dist_Sup_to_Warehouse[j, k] * VehicleCostPerMileAndTonneOverall[v_type] / 1000

Base_CostWarehouseCustomer = np.zeros_like(Dist_Warehouse_to_Cust)
for j in Groups:
    for i in Customer:
        Base_CostWarehouseCustomer[j, i] = 2 * Dist_Warehouse_to_Cust[j, i] * VehicleCostPerMileAndTonneOverall[3] / 1000

# =============================================================================
# 3. Define Sensitivity Analysis Scenarios
# =============================================================================
# Precisely define multipliers for each scenario
scenarios = [
    {"name": "1. Base Case",              "alpha": 1.0, "beta": 1.0, "gamma": 1.0, "delta": 1.0},
    {"name": "2. Transport Shock (+50%)", "alpha": 1.5, "beta": 1.0, "gamma": 1.0, "delta": 1.0},
    {"name": "2. Transport Shock (+100%)","alpha": 2.0, "beta": 1.0, "gamma": 1.0, "delta": 1.0},
    {"name": "3. Facility Inflation (+50%)","alpha":1.0, "beta": 1.5, "gamma": 1.0, "delta": 1.0},
    {"name": "3. Facility Inflation (+100%)","alpha":1.0, "beta": 2.0, "gamma": 1.0, "delta": 1.0},
    {"name": "4. Capacity Squeeze (70%)", "alpha": 1.0, "beta": 1.0, "gamma": 0.7, "delta": 1.0},
    {"name": "4. Capacity Squeeze (50%)", "alpha": 1.0, "beta": 1.0, "gamma": 0.5, "delta": 1.0},
    {"name": "5. Supplier Disruption (80%)","alpha": 1.0,"beta": 1.0, "gamma": 1.0, "delta": 0.8},
    {"name": "5. Supplier Disruption (50%)","alpha": 1.0,"beta": 1.0, "gamma": 1.0, "delta": 0.5}
]

# List to store results for each scenario
results_summary = []

# Disable verbose Xpress solver output to avoid flooding the console
xp.setOutputEnabled(False) 

# =============================================================================
# 4. Automated Optimization Loop
# =============================================================================
for scen in scenarios:
    name = scen["name"]
    alpha, beta, gamma, delta = scen["alpha"], scen["beta"], scen["gamma"], scen["delta"]
    print(f"Solving Scenario: {name} ... ", end="")

    # --- Step A: Apply Sensitivity Multipliers ---
    Scen_CostSupplierWarehouse = Base_CostSupplierWarehouse * alpha
    Scen_CostWarehouseCustomer = Base_CostWarehouseCustomer * alpha
    Scen_TotalSetupCost = Base_TotalSetupCost * beta
    Scen_OperatingCost = Base_OperatingCost * beta
    Scen_TotalCapacity = Base_TotalCapacity * gamma
    Scen_supplier_cap_matrix = Base_supplier_cap_matrix * delta

    # --- Step B: Create a New Model (Must be inside loop to avoid contamination) ---
    prob = xp.problem(name)

    # Variables
    y = prob.addVariables(Groups, Period, name='y', vartype=xp.binary)
    z = prob.addVariables(Groups, Period, name='z', vartype=xp.binary)
    x = prob.addVariables(Customer, Groups, Period, name='x', vartype=xp.binary)
    M = prob.addVariables(Supplier, Groups, Period, Products, vartype=xp.integer, name="M")

    # Objective
    # [Bug Fix 2] Corrected demand matrix indexing to demand_matrices[t+1][q, i] (previously q-1, i-1)
    obj_eqn = xp.Sum(z[j, t] * Scen_TotalSetupCost[j] +
                     y[j, t] * Scen_OperatingCost[j] +
                     xp.Sum(Scen_CostSupplierWarehouse[j, k] * M[k, j, t, q] for k in Supplier) +
                     xp.Sum(Scen_CostWarehouseCustomer[j, i] * demand_matrices[t+1][q, i] * x[i, j, t] for i in Customer)
                     for j in Groups for t in Period for q in Products)
    prob.setObjective(obj_eqn, sense=xp.minimize)

    # Constraints (using Scen_ parameters)
    prob.addConstraint(y[j, t] >= y[j, t-1] for j in Groups for t in Period if t > 0)
    prob.addConstraint(x[i, j, t] <= y[j, t] for i in Customer for j in Groups for t in Period)
    prob.addConstraint(xp.Sum(z[j, t] for t in Period) <= 1 for j in Groups)
    prob.addConstraint(xp.Sum(x[i, j, t] for j in Groups) == 1 for i in Customer for t in Period)
    prob.addConstraint(M[k,j,t,q] == 0 for k in Supplier for q in Products if q+1 != Product_Sup[k] for j in Groups for t in Period)
    prob.addConstraint(y[j,0] == z[j,0] for j in Groups)
    prob.addConstraint(y[j,t] - y[j,t-1] == z[j,t] for j in Groups for t in Period if t > 0)
    
    # Supplier Capacity Constraint (using delta-adjusted capacity)
    prob.addConstraint(y[j,t] * Scen_supplier_cap_matrix[k,q] >= M[k, j, t, q] for k in Supplier for j in Groups for t in Period for q in Products)
    prob.addConstraint(M[k, j, t, q] >= 0 for k in Supplier for j in Groups for t in Period for q in Products)

    # Flow & Warehouse Capacity Constraint (using gamma-adjusted capacity)
    prob.addConstraint(xp.Sum(demand_matrices[t+1][q, i] * x[i,j,t] for i in Customer) <= xp.Sum(M[k,j,t,q] for k in Supplier) for j in Groups for q in Products for t in Period)
    prob.addConstraint(xp.Sum(M[k, j, t, q] for k in Supplier for q in Products) <= Scen_TotalCapacity[j] * y[j, t] for j in Groups for t in Period)

    # --- Step C: Solve and Extract Results ---
    prob.solve()
    
    if prob.attributes.solstatus in [xp.SolStatus.OPTIMAL, xp.SolStatus.FEASIBLE]:
        total_cost = prob.attributes.objval
        
        # Track detailed annual facility opening plan
        build_schedule = {}  # Dictionary recording build actions by year
        num_opened = 0       # Total number of facilities opened
        
        for t in Period:
            # Extract warehouse IDs built in year t (z > 0.5)
            built_this_year = [j for j in Groups if prob.getSolution(z[j, t]) > 0.5]
            if built_this_year:
                build_schedule[f"Year {t+1}"] = built_this_year
                num_opened += len(built_this_year)
        
        # Convert dictionary to string for tabular storage
        schedule_str = str(build_schedule) if build_schedule else "No facilities built"
        
        print(f"Done! Cost: £{total_cost:,.0f} | Opened: {num_opened} | Schedule: {schedule_str}")
        
        results_summary.append({
            "Scenario": name,
            "Total Cost (£)": total_cost,
            "Total Facilities Opened": num_opened,
            "Build Schedule (Year: [IDs])": schedule_str,
            "Status": "Optimal" if prob.attributes.solstatus == xp.SolStatus.OPTIMAL else "Feasible"
        })
    else:
        print("Failed (Infeasible or Unbounded)")
        results_summary.append({
            "Scenario": name,
            "Total Cost (£)": "N/A",
            "Total Facilities Opened": "N/A",
            "Build Schedule (Year: [IDs])": "N/A",
            "Status": "Infeasible"
        })

# =============================================================================
# 5. Output Summary Report
# =============================================================================
df_results = pd.DataFrame(results_summary)
print("\n" + "="*80)
print("SENSITIVITY ANALYSIS RESULTS SUMMARY")
print("="*80)
print(df_results.to_string(index=False))

# Save results as CSV for reporting
df_results.to_csv("Sensitivity_Analysis_Results.csv", index=False)
